In [1]:
import modules.text_processing
import torch
import torch.nn.functional as F
from imp import reload
from modules.text_processing import read_split_file_lyrics, BatchWrapper, generate_iterators_lyrics


In [2]:
#Read and split data
train_path, val_path = read_split_file_lyrics("Data/lyrics_cleaned.csv","tmp/")

In [3]:
#Create iterators
torch.cuda.empty_cache() 
batch_size = 10
train_iter, val_iter, TEXT = generate_iterators_lyrics(train_path, val_path, 
                                                           batch_size = batch_size, device="cuda")



In [4]:
#Taken and modified from lab/hw
def evaluate(model, data_iter, batch_size):
    model.eval()
    val_loss = 0
    total = 0
    total_acc = 0
    val_acc = 0
    for i in range(len(data_iter)):
        vectors,labels = next(data_iter.__iter__())
        output = model(vectors)
        #_, predicted = torch.topk(output.data, k=2, dim=1)
        #predictions = torch.zeros(labels.size()).to("cuda")
        #predictions.scatter_(1, predicted, 1)
        val_acc += ((output > 0.5) == labels.byte()).sum().item()
        total_acc += labels.size(0)*2
        val_loss += F.kl_div(output.log(), labels)
        total +=1
    return val_loss / total, val_acc / total_acc

In [5]:
#Taken and modified from lab/hw
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter, verbose=True):
    
    epoch = 0
    total_batches = int(len(training_iter))
    dev_accuracies = []
    test_accuracies = []
    while epoch <= num_epochs:
        print("Training...")
        for i in range(total_batches):
            model.train()
            vectors, labels = next(training_iter.__iter__())
            model.zero_grad()
            #print(vectors)
            #print(labels)
            output = model(vectors)
            lossy = loss_(output.log(), labels)
            lossy.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optim.step()


        model.eval()
        print("Evaluating dev...")
        eval_loss, eval_acc = evaluate(model, dev_iter, batch_size)
        dev_accuracies.append(eval_acc)
        if verbose:
            print("Epoch %i; Loss %f; Dev loss %f, Dev acc: %f"  %(epoch, lossy.item(), eval_loss, eval_acc))
        epoch += 1    
    best_dev = max(dev_accuracies)
    best_test = test_accuracies[np.argmax(dev_accuracies)]
    return best_dev, best_test

In [6]:
import numpy as np
import pandas as pd
import modules.models
reload(modules.models)
from modules.models import BiLSTMBaseline

dropout_min = 0.2
dropout_max = 0.8

learning_rate_min = 0.0001
learning_rate_max = 0.01

hidden_dim_min = 50
hidden_dim_max = 300

v_dim_min = 5
v_dim_max = 30
label_size = 2
embedding_dim = 300

dropouts = []
learning_rates = []
hidden_dims = []
v_dims = []
dev_accuracies = []
test_accuracies = []

for i in range(60):
    print("Iteration: ", i)
    #Create wrappers for iterators
    train_batch = BatchWrapper(train_iter, "lyrics", ["pos", "neg"])
    valid_batch = BatchWrapper(val_iter, "lyrics", ["pos", "neg"])
    
    dropout = np.random.uniform(dropout_min, dropout_max)
    learning_rate = 10**np.random.uniform(np.log10(learning_rate_min), np.log10(learning_rate_max))
    hidden_dim = np.random.randint(hidden_dim_min, hidden_dim_max)
    v_dim = np.random.randint(v_dim_min, v_dim_max)
    
    m = BiLSTMBaseline(vocab_size = len(TEXT.vocab), embedding_dim = embedding_dim, hidden_dim = hidden_dim, label_size=label_size, 
                       v_dim = v_dim, pretrained_vec=TEXT.vocab.vectors, use_gpu = True, dropout = dropout)
    m.to("cuda")
    opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), learning_rate)
    dev_acc, test_acc = training_loop(model=m, training_iter=train_batch, dev_iter=valid_batch, loss_=F.kl_div, optim=opt, num_epochs=10, batch_size = batch_size)

    dropouts.append(dropout)
    learning_rates.append(learning_rate)
    hidden_dims.append(hidden_dim)
    v_dims.append(v_dim)
    dev_accuracies.append(dev_acc)
    test_accuracies.append(test_acc)
    df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, "hidden_dim": hidden_dims, "v_dim": v_dims, "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})
    df.to_csv("lyrics_results_test.csv")

    
df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, "hidden_dim": hidden_dims, "v_dim": v_dims, "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})

    


Iteration:  0
Training...


/home/philip/.local/lib/python3.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


Evaluating dev...
Epoch 0; Loss 0.321692; Dev loss 0.331445, Dev acc: 0.565000
Training...
Evaluating dev...
Epoch 1; Loss 0.316250; Dev loss 0.294673, Dev acc: 0.615000
Training...
Evaluating dev...
Epoch 2; Loss 0.296457; Dev loss 0.302922, Dev acc: 0.650000
Training...
Evaluating dev...
Epoch 3; Loss 0.280436; Dev loss 0.249284, Dev acc: 0.805000
Training...
Evaluating dev...
Epoch 4; Loss 0.258112; Dev loss 0.287795, Dev acc: 0.730000
Training...
Evaluating dev...
Epoch 5; Loss 0.191682; Dev loss 0.300858, Dev acc: 0.725000
Training...
Evaluating dev...
Epoch 6; Loss 0.178601; Dev loss 0.236032, Dev acc: 0.810000
Training...
Evaluating dev...
Epoch 7; Loss 0.197182; Dev loss 0.320615, Dev acc: 0.690000
Training...
Evaluating dev...
Epoch 8; Loss 0.205936; Dev loss 0.317258, Dev acc: 0.705000
Training...
Evaluating dev...
Epoch 9; Loss 0.125670; Dev loss 0.291099, Dev acc: 0.720000
Training...
Evaluating dev...
Epoch 10; Loss 0.092259; Dev loss 0.241016, Dev acc: 0.780000


IndexError: list index out of range

In [ ]:
df.sort_values("dev_accuracy")